In [1]:
from functions import (prep, folding, inbu, LSTM_model,setting)
import numpy as np
from matplotlib import pyplot as plt
from keras.models import load_model
from jellyfish import damerau_levenshtein_distance, levenshtein_distance
import pandas as pd
import distance
from sklearn import metrics
np.random.seed(42)
%load_ext autoreload
%autoreload 2

In [2]:
setting_inputs = setting.inputdef()
csvname = f"{setting_inputs['flatten_by']}_complete" if setting_inputs['complete'] else f"{setting_inputs['flatten_by']}_filter"
# Define model_file based on single_log value
model_file = f"{csvname}_single" if setting_inputs['single_log'] else f"{csvname}_enriched"

Settings:
flatten_by: Items
single_log: False
complete: True
Add customer: False
Normalize: True
Position excluded: False



In [3]:
setting_inputs

{'source': 'running-example',
 'add_customer': 0,
 'normalize': True,
 'pos_ex': False,
 'flatten_by': 'Items',
 'single_log': False,
 'complete': True}

In [4]:
# prep the ocel and reading
ocel, act_dict, cust_dict = prep.prep_ocel_complete(setting_inputs=setting_inputs,csvname=csvname)
ocel_train, ocel_test = folding.folding_train_test(ocel, csvname= model_file)
## define some static variables 
divisor = np.mean(ocel['Time_Diff'])  # average time between events
divisor2 = np.mean(ocel['Time_Since_Start'])  # average time between current and first events
divisorTR = np.mean(ocel['Remaining_Time'])  # average time instance remaining
divisor3 = ocel.groupby('Case_ID')['Time_Since_Start'].apply(lambda x: (x.iloc[-1] - x).mean()).mean()

print(f"\ndivisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
print(f"divisor3: {divisor3}")
print(f'Amount of rows of the OCEL: {len(ocel)}\n')

num_of_features, max_trace_length, act_feat,cust_feat, target_act_feat, target_act_feat_dict, other_features = setting.feature_dimensios(ocel=ocel,setting_input=setting_inputs)
number_of_train_cases = len(ocel_train)
target_act_length = len(target_act_feat)
print(f"Number of train cases: {number_of_train_cases}, Max trace length: {max_trace_length}, Number of features: {num_of_features}\n")


{'place order': 'A', 'pick item': 'B', 'confirm order': 'C', 'item out of stock': 'D', 'reorder item': 'E', 'pay order': 'F', 'create package': 'G', 'send package': 'H', 'failed delivery': 'I', 'package delivered': 'J', 'payment reminder': 'K'}
{'Marco Pegoraro': 'a', 'Gyunam Park': 'b', 'Majid Rafiei': 'c', 'Junxiong Gao': 'd', 'Seran Uysal': 'e', 'Christina Rensinghof': 'f', 'Wil van der Aalst': 'g', 'Christine Dobbert': 'h', 'Luis Santos': 'i', 'Kefang Ding': 'j', 'Mohammadreza Fani Sani': 'k', 'Tobias Brockhoff': 'l', 'Anahita Farhang Ghahfarokhi': 'm', 'Mahnaz Qafari': 'n', 'Claudia Graf': 'o', 'Mahsa Bafrani': 'p', 'Lisa Mannel': 'q'}

divisor: 165106.43637259887
divisor2: 556805.5647981268
divisorTR: 850395.9780488932
divisor3: 793350.2040861561
Amount of rows of the OCEL: 64917

Number of train cases: 43289, Max trace length: 16, Number of features: 18



In [5]:
X_train,y_train_a, y_train_t, y_train_tr = inbu.generating_inputs(ocel_fold=ocel_train,ocel=ocel,setting_input=setting_inputs,
                                                                  dn= divisor, ds= divisor2, dr= divisorTR)

print(f"Shape of X_train: {X_train.shape}")
print(f"This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): {(number_of_train_cases, max_trace_length, num_of_features)} => {X_train.shape ==(number_of_train_cases, max_trace_length, num_of_features)}")
print(f"Shape of y_train_a: {y_train_a.shape}, this matches the desired shape (number_of_train_cases, target_act_length): {(number_of_train_cases, target_act_length)} => {y_train_a.shape ==(number_of_train_cases, target_act_length)}")
print(f"Shape of y_train_t: {y_train_t.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_t.shape ==(number_of_train_cases, )}")
print(f"Shape of y_train_tr: {y_train_tr.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_tr.shape ==(number_of_train_cases, )}\n")


Shape of X_train: (43289, 16, 18)
This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): (43289, 16, 18) => True
Shape of y_train_a: (43289, 12), this matches the desired shape (number_of_train_cases, target_act_length): (43289, 12) => True
Shape of y_train_t: (43289,), this matches the desired shape (number_of_train_cases, ): (43289,) => True
Shape of y_train_tr: (43289,), this matches the desired shape (number_of_train_cases, ): (43289,) => True



In [ ]:
print(f'For the following setting a model is now trained {model_file}\n')
history, best_model_name, early_stopping= LSTM_model.LSTM_MODEL(X_train, y_train_a, y_train_t, y_train_tr,filename=model_file)

In [ ]:
val_loss = min(history.history['val_loss'])
val_loss2 = best_model_name.best
epoch = early_stopping.stopped_epoch - 49
print(f'The best value for the validation loss is  {val_loss} and was archived at the epoch {epoch}\n')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

modelname = 'model_' + model_file + f"_{epoch:02d}-{val_loss:.2f}.h5"

print(f'The best model has the name {modelname}\n')

In [69]:
model = load_model(f'./output_files/models/{modelname}')
X_test, y_test_a, y_test_t, y_test_tr = inbu.generating_inputs(ocel_fold=ocel_test,ocel=ocel,setting_input=setting_inputs,
                                                                  dn= divisor, ds= divisor2, dr= divisorTR)

# y_t = y_t * divisor3
y = model.predict(X_test,verbose=1)
y_char = y[0][:][:]
y_t = y[1][:][:]
y_tr = y[2][:][:]
max_index_list = [np.argmax(pred) for pred in y_char]
pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
y_t = np.maximum(y_t, 0)
y_t1 = y_t * divisor
y_tr1 = y_tr * divisorTR

columns_to_drop = [col for col in ocel_test.columns if 'Act_' in col] + \
                [col for col in ocel_test.columns if 'Cust_' in col] + \
                ['Customers', 'Next_Time_Since_Start',
                'Next_Time_Since_Midnight', 'Next_Weekday',
                'Position', 'Time_Since_Midnight', 'Weekday'] + other_features

columns_to_drop_existing = [col for col in columns_to_drop if col in ocel_test.columns]
output_ocel = ocel_test.drop(columns=columns_to_drop_existing).copy()
output_ocel['Pred_Activity'] = pred_act_list
output_ocel['Pred_Time_Diff'] = y_t1
output_ocel['Pred_Remaining_Time'] = y_tr1

output_ocel['Levenshtein'] = output_ocel.apply(lambda row: 1 - levenshtein_distance(row['Pred_Activity'], row['Next_Activity']), axis=1)
output_ocel['Damerau'] = output_ocel.apply(lambda row: 1 - (damerau_levenshtein_distance(row['Pred_Activity'], row['Next_Activity']) / max(len(row['Pred_Activity']), len(row['Next_Activity']))), axis=1)
output_ocel['Damerau'] = output_ocel['Damerau'].clip(lower=0)
output_ocel['Jaccard'] = output_ocel.apply(lambda row: 1 - distance.jaccard(row['Pred_Activity'], row['Next_Activity']), axis=1)

act_comp = output_ocel['Pred_Activity'] == output_ocel['Next_Activity'] 
print(f'The accuracy of the activation prediction is {sum(act_comp)/len(act_comp)}')
MAE_Time_diff = metrics.mean_absolute_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60),output_ocel['Next_Time_Diff']/ (24 * 60 * 60))
MAE_rem_time = metrics.mean_absolute_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60),output_ocel['Next_Remaining_Time']/ (24 * 60 * 60))
RMSE_Time_diff = metrics.mean_squared_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60),output_ocel['Next_Time_Diff']/ (24 * 60 * 60),squared=False)
RMSE_rem_time = metrics.mean_squared_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60),output_ocel['Next_Remaining_Time']/ (24 * 60 * 60),squared=False)
print(f'MAE of the time between events in days {MAE_Time_diff}')
print(f'MAE of the remaining time in days {MAE_rem_time}')
# print(f'RMSE of the time between events in days {RMSE_Time_diff}')
# print(f'RMSE of the remaining time in days {RMSE_rem_time}')

676/676 [==============================] - 5s 6ms/step
The accuracy of the activation prediction is 0.7942481967819494
MAE of the time between events in days 1.3075972576665165
MAE of the remaining time in days 5.281885940627627


In [22]:
modelname = 'model_Items_complete_enriched_24-1.73.h5'

model = load_model(f'./output_files/models/{modelname}')
prefix_length = 2
idx = (ocel_test['Trace_Len'].values >= prefix_length) *(ocel_test['Position'].values == prefix_length)
len(ocel_test.loc[idx,'Case_ID'].unique())

2719

In [19]:
prefix_lengths = range(2, max_trace_length)  # List of prefix lengths to consider
results = []
print(max_trace_length)
for prefix_length in prefix_lengths:    
    if setting.feature_dimensios(ocel=ocel_test,setting_input=setting_inputs)[1] == prefix_length:
        continue
    print(f"Results for Prefix Length {prefix_length}:")
    # Generate inputs with the current prefix length
    X_test, y_test_a, y_test_t, y_test_tr = inbu.generating_inputs_pref(ocel_fold=ocel_test, ocel=ocel,
                                                                  setting_input=setting_inputs,
                                                                  dn=divisor, ds=divisor2, dr=divisorTR,
                                                                  prefix_length=prefix_length)
    # Make predictions with the model
    y = model.predict(X_test, verbose=1)
    y_char, y_t, y_tr = y[0][:][:], y[1][:][:], y[2][:][:]

    max_index_list = [np.argmax(pred) for pred in y_char]
    pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
    y_t = np.maximum(y_t, 0)
    y_t, y_tr = y_t * divisor, y_tr * divisorTR
    y_test_t, y_test_tr = y_test_t * divisor, y_test_tr * divisorTR  
    columns_to_drop = [col for col in ocel_test.columns if 'Act_' in col] + \
                    [col for col in ocel_test.columns if 'Cust_' in col] + \
                    ['Customers', 'Next_Time_Since_Start',
                    'Next_Time_Since_Midnight', 'Next_Weekday',
                    'Position', 'Time_Since_Midnight', 'Weekday'] + other_features
    idx = (ocel_test['Trace_Len'].values >= prefix_length) * (ocel_test['Position'].values == prefix_length)
    next_act = ocel_test.loc[idx,'Next_Activity'] 
    act_comp = pred_act_list == next_act
    mae_time_diff = metrics.mean_absolute_error(y_t / (24 * 60 * 60), y_test_t / (24 * 60 * 60))
    mae_remaining_time = metrics.mean_absolute_error(y_tr / (24 * 60 * 60), y_test_tr / (24 * 60 * 60))
    rmse_time_diff = metrics.mean_squared_error(y_t / (24 * 60 * 60), y_test_t / (24 * 60 * 60), squared=False)
    rmse_remaining_time = metrics.mean_squared_error(y_tr / (24 * 60 * 60), y_test_tr / (24 * 60 * 60), squared=False)
    # Store the results for the current prefix length
    results.append({
        'Prefix Length': prefix_length,
        'length': sum(idx),
        'Accuracy Activity': sum(act_comp)/len(act_comp),
        'MAE Time Difference': mae_time_diff,
        'MAE Remaining Time': mae_remaining_time,
        'RMSE Time Difference': rmse_time_diff,
        'RMSE Remaining Time': rmse_remaining_time
    })

# Output the overall results
df_results = pd.DataFrame(results)
print("\nOverall Results:")
display(df_results)

16
Results for Prefix Length 2:
85/85 [==============================] - 0s 5ms/step
Results for Prefix Length 3:
85/85 [==============================] - 0s 5ms/step
Results for Prefix Length 4:
85/85 [==============================] - 0s 5ms/step
Results for Prefix Length 5:
85/85 [==============================] - 0s 5ms/step
Results for Prefix Length 6:
85/85 [==============================] - 0s 5ms/step
Results for Prefix Length 7:
85/85 [==============================] - 0s 5ms/step
Results for Prefix Length 8:
42/42 [==============================] - 0s 6ms/step
Results for Prefix Length 9:
26/26 [==============================] - 0s 6ms/step
Results for Prefix Length 10:
9/9 [==============================] - 0s 6ms/step
Results for Prefix Length 11:
4/4 [==============================] - 0s 5ms/step
Results for Prefix Length 12:
2/2 [==============================] - 0s 4ms/step
Results for Prefix Length 13:
1/1 [==============================] - 0s 24ms/step
Results for Pref

,Prefix Length,length,Accuracy Activity,MAE Time Difference,MAE Remaining Time,RMSE Time Difference,RMSE Remaining Time
0,2,2719,0.675616,1.031804,8.186174,1.702078,12.958726
1,3,2719,0.621552,1.129150,7.586163,1.746500,12.256253
2,4,2719,0.657227,1.275151,6.995411,2.618401,11.480614
3,5,2719,0.745495,1.373128,6.074540,3.523567,10.603509
4,6,2719,0.658330,2.794785,3.439087,5.050892,8.160406
5,7,2719,0.776021,1.641297,1.567283,3.617359,5.220357
6,8,1326,0.788839,1.482172,1.520387,3.339675,5.132291
7,9,830,0.836145,1.255064,0.882720,3.308374,3.996043
8,10,282,0.822695,1.429171,0.750838,3.477728,3.581691
9,11,107,0.869159,1.134443,0.599822,3.063836,2.517212


In [26]:
df_results.mean()

Prefix Length              8.000000
length                  1454.538462
Accuracy Activity          0.794879
MAE Time Difference        1.213390
MAE Remaining Time         2.899317
RMSE Time Difference       2.690304
RMSE Remaining Time        5.857893
dtype: float64

In [25]:
prefix_lengths = range(2, max_trace_length)  # List of prefix lengths to consider

MAE_Rem = []
act_acc = []
MAE_next = []

print(max_trace_length)
for prefix_length in prefix_lengths:    
    if setting.feature_dimensios(ocel=ocel_test,setting_input=setting_inputs)[1] == prefix_length:
        continue
    print(f"Results for Prefix Length {prefix_length}:")
    # Generate inputs with the current prefix length
    X_test, y_test_a, y_test_t, y_test_tr = inbu.generating_inputs_pref(ocel_fold=ocel_test, ocel=ocel,
                                                                  setting_input=setting_inputs,
                                                                  dn=divisor, ds=divisor2, dr=divisorTR,
                                                                  prefix_length=prefix_length)
    # Make predictions with the model
    y = model.predict(X_test, verbose=1)
    y_char, y_t, y_tr = y[0][:][:], y[1][:][:], y[2][:][:]

    max_index_list = [np.argmax(pred) for pred in y_char]
    pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
    y_t = np.maximum(y_t, 0)
    y_t, y_tr = y_t * divisor, y_tr * divisorTR
    y_test_t, y_test_tr = y_test_t * divisor, y_test_tr * divisorTR  

    idx = (ocel_test['Trace_Len'].values >= prefix_length) * (ocel_test['Position'].values == prefix_length)

    next_act = ocel_test.loc[idx,'Next_Activity'] 
    act_comp = pred_act_list == next_act
    mae_time_diff = [np.abs(x/ (24 * 60 * 60) -  y/ (24 * 60 * 60)) for x,y in zip(y_t,y_test_t)]
    mae_remaining_time = [np.abs(x / (24 * 60 * 60) - y / (24 * 60 * 60)) for x,y in zip(y_tr,y_test_tr)]
    # rmse_time_diff = [np.abs(x / (24 * 60 * 60)-  y / (24 * 60 * 60), squared=False) for x,y in zip(y_t,y_test_t)]
    # rmse_remaining_time = [np.abs(x / (24 * 60 * 60) -  y / (24 * 60 * 60), squared=False) for x,y in zip(y_t,y_test_t)]
    # Store the results for the current prefix length
    act_acc.extend(act_comp)
    MAE_next.extend(mae_time_diff)
    MAE_Rem.extend(mae_remaining_time)


print(sum(act_acc)/len(act_acc))
print(sum(MAE_next)/len(act_acc))
print(sum(MAE_Rem)/len(act_acc))


16
Results for Prefix Length 2:
85/85 [==============================] - 0s 6ms/step
Results for Prefix Length 3:
85/85 [==============================] - 0s 6ms/step
Results for Prefix Length 4:
85/85 [==============================] - 1s 6ms/step
Results for Prefix Length 5:
85/85 [==============================] - 0s 6ms/step
Results for Prefix Length 6:
85/85 [==============================] - 1s 6ms/step
Results for Prefix Length 7:
85/85 [==============================] - 0s 6ms/step
Results for Prefix Length 8:
42/42 [==============================] - 0s 6ms/step
Results for Prefix Length 9:
26/26 [==============================] - 0s 6ms/step
Results for Prefix Length 10:
9/9 [==============================] - 0s 6ms/step
Results for Prefix Length 11:
4/4 [==============================] - 0s 6ms/step
Results for Prefix Length 12:
2/2 [==============================] - 0s 4ms/step
Results for Prefix Length 13:
1/1 [==============================] - 0s 19ms/step
Results for Pref

In [17]:
print(sum(act_acc)/len(act_acc))
print(sum(MAE_next)/len(MAE_next))
print(sum(MAE_Rem)/len(MAE_Rem))

0.46787481540972564
[1.89392715]
[5.0078172]


In [ ]:
import pickle
modelname = ""
# Load the saved history from file
with open(f'/ouput_files/history/{modelname}_history.pkl', 'rb') as file:
    loaded_history = pickle.load(file)

# Access the loaded history
print(loaded_history)
plt.plot(loaded_history['loss'])
plt.plot(loaded_history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()